In [25]:
# Combine the following two DataFrames

# Importing Libraries
import pandas as pd
from datasets import load_dataset
import matplotlib.pyplot as plt

# Loading Data
dataset = load_dataset('lukebarousse/data_jobs')
df = dataset['train'].to_pandas()

# Data Cleanup
df['job_posted_date'] = pd.to_datetime(df['job_posted_date'])

In [26]:
# Example
# Calculate the projected salary for next year, assuming 3.0% rate for all role.

def inflation(salary):
    return salary * 1.03

df['salary_year_inflated'] = df['salary_year_avg'].apply(inflation)

df[pd.notna(df['salary_year_avg'])][['salary_year_avg', 'salary_year_inflated']]

,salary_year_avg,salary_year_inflated
28,109500.0,112785.00
77,140000.0,144200.00
92,120000.0,123600.00
100,228222.0,235068.66
109,89000.0,91670.00
...,...,...
785624,139216.0,143392.48
785641,150000.0,154500.00
785648,221875.0,228531.25
785682,157500.0,162225.00


In [27]:
df['salary_year_inflated'] = df['salary_year_avg'].apply(lambda salary: salary * 1.03)
df[pd.notna(df['salary_year_avg'])][['salary_year_avg', 'salary_year_inflated']]

,salary_year_avg,salary_year_inflated
28,109500.0,112785.00
77,140000.0,144200.00
92,120000.0,123600.00
100,228222.0,235068.66
109,89000.0,91670.00
...,...,...
785624,139216.0,143392.48
785641,150000.0,154500.00
785648,221875.0,228531.25
785682,157500.0,162225.00


In [28]:
# Example 2
# Calculate projected salaries next year, but:

# For senior roles (e.g., Senior Data Analysts), assume the rate is 5%
# For all other roles, assume rate is 3%

def pro_salary(row):
    if "Senior" in row['job_title_short']:
        return 1.05 * row['salary_year_avg']
    else:
        return 1.03 * row['salary_year_avg']
    
df['salary_year_inflated'] = df.apply(pro_salary, axis=1)
df[pd.notna(df['salary_year_avg'])][['job_title_short', 'salary_year_avg', 'salary_year_inflated']]

,job_title_short,salary_year_avg,salary_year_inflated
28,Data Scientist,109500.0,112785.00
77,Data Engineer,140000.0,144200.00
92,Data Engineer,120000.0,123600.00
100,Data Scientist,228222.0,235068.66
109,Data Analyst,89000.0,91670.00
...,...,...,...
785624,Data Engineer,139216.0,143392.48
785641,Data Engineer,150000.0,154500.00
785648,Data Scientist,221875.0,228531.25
785682,Data Scientist,157500.0,162225.00


In [29]:
import ast

df['job_skills'] = df['job_skills'].apply(lambda y: ast.literal_eval(y) if pd.notna(y) else y)
df['job_skills']

0                                                      None
1                [r, python, sql, nosql, power bi, tableau]
2         [python, sql, c#, azure, airflow, dax, docker,...
3         [python, c++, java, matlab, aws, tensorflow, k...
4         [bash, python, oracle, aws, ansible, puppet, j...
                                ...                        
785736    [bash, python, perl, linux, unix, kubernetes, ...
785737                               [sas, sas, sql, excel]
785738                                  [powerpoint, excel]
785739    [python, go, nosql, sql, mongo, shell, mysql, ...
785740                                          [aws, flow]
Name: job_skills, Length: 785741, dtype: object

## Problems

In [32]:
# 1_Problem Statement:
# Convert the job_posted_date column to a string format 'YYYY-MM-DD' and create a new column job_posted_date_str.

df['job_posted_date_str'] = df['job_posted_date'].apply(lambda y: y.strftime('%Y-%m-%d'))
df[['job_posted_date', 'job_posted_date_str']].head()

,job_posted_date,job_posted_date_str
0,2023-06-16 13:44:15,2023-06-16
1,2023-01-14 13:18:07,2023-01-14
2,2023-10-10 13:14:55,2023-10-10
3,2023-07-04 13:01:41,2023-07-04
4,2023-08-07 14:29:36,2023-08-07


In [36]:
# 2_Problem Statement:
# Calculate the number of days since each job was posted. Create a new column days_since_posted that contains this value. 
# Use the job_posted_date column.
# Note: You need to import the datetime library and use the datetime module to get the current date using .now().

from datetime import datetime

current_date = datetime.now()
df['days_since_posted'] = df['job_posted_date'].apply(lambda date: (current_date - date).days)
df[['job_posted_date', 'days_since_posted']].head()


,job_posted_date,days_since_posted
0,2023-06-16 13:44:15,413
1,2023-01-14 13:18:07,566
2,2023-10-10 13:14:55,297
3,2023-07-04 13:01:41,395
4,2023-08-07 14:29:36,361


In [39]:
# 3_Problem Statement:
# Create a copy of the DataFrame called df_filtered and drop the NaN values for salary_year_avg.
# Then, create a new column salary_category that categorizes the salary_year_avg into three categories:
# 'Low' for salaries less than 60,000, 'Medium' for salaries between 60,000 and 100,000, and 'High' for salaries greater than 100,000.
# Then show the df_filtered DataFrame and the salary_year_avg and salary_category columns.

df_filtered = df.dropna(subset=['salary_year_avg']).copy()
df_filtered['salary_category'] = df['salary_year_avg'].apply(lambda salary: 'Low' if salary < 60000 else 'Medium' if salary <= 100000 else 'High')
df_filtered[['salary_year_avg', 'salary_category']]

,salary_year_avg,salary_category
28,109500.0,High
77,140000.0,High
92,120000.0,High
100,228222.0,High
109,89000.0,Medium
...,...,...
785624,139216.0,High
785641,150000.0,High
785648,221875.0,High
785682,157500.0,High


In [40]:
def category(salary):
    if salary < 60000:
        return 'Low'
    elif salary <=100000:
        return 'Medium'
    else:
        return 'High'
    
df_filtered = df.dropna(subset=['salary_year_avg']).copy()
df_filtered['salary_category'] = df['salary_year_avg'].apply(category)
df_filtered[['salary_year_avg', 'salary_category']]

,salary_year_avg,salary_category
28,109500.0,High
77,140000.0,High
92,120000.0,High
100,228222.0,High
109,89000.0,Medium
...,...,...
785624,139216.0,High
785641,150000.0,High
785648,221875.0,High
785682,157500.0,High
